In [1]:
import pandas as pd
import numpy as np
from io import StringIO

In [121]:
def squeeze_nan_left(x):
            original_columns = x.index.tolist()
            squeezed = x.dropna()
            squeezed.index = [original_columns[n] for n in range(squeezed.count())]
            return squeezed.reindex(original_columns, fill_value=np.nan)

In [137]:
def SCT_table_format(SCT):
            #SCT = SCT.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 

            for i in range(0,SCT.shape[0]):
                for j in range(0,SCT.shape[1]):
                    if type(SCT.iloc[i,j]) == str:
                        if SCT.iloc[i,j] == "-" or SCT.iloc[i,j] == "—":
                            SCT.iloc[i,j] = 0
                        elif '(' in SCT.iloc[i,j]:
                            tempstr = SCT.iloc[i,j].split("(")
                            tempstr2 = tempstr[0]
                            SCT.iloc[i,j] = tempstr2
                        elif len(SCT.iloc[i,j]) < 2:
                            SCT.iloc[i,j] = np.nan             

            SCT = SCT.replace('', np.nan)
            SCT.iloc[:,0] = SCT.iloc[:,0].fillna(value = '')
            SCT = SCT.fillna(value = '')

            for i in range(0,SCT.shape[0]):
                if SCT.iloc[i, SCT.shape[1]-1] == '':
                    rightslip = ['',SCT.iloc[i]]
                    rightslip = rightslip[:-1]
                    SCT.iloc[i] = rightslip

            SCT.replace('',np.nan,inplace = True)
            SCT = SCT.dropna()

            SCT = SCT.rename(columns=SCT.iloc[0]).drop(index = 1)

            return SCT

In [141]:
#Everi - https://www.sec.gov/Archives/edgar/data/1318568/000155837017002779/evri-20170523xdef14a.htm
#MNTA - https://www.sec.gov/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm

url = 'https://www.sec.gov/Archives/edgar/data/1410939/000104746917002856/a2231921zdef14a.htm#ds46201_summary_compensation_table'

filing = pd.read_html(url)
#filing[:]
#168 for Everi
#128 for MNTA
#20 for OPHT

sctind = 20
SCTo = filing[sctind]
SCTo

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name and principal position,NaN,Year,NaN,Salary ($),NaN,Bonus ($)(1),NaN,Stock Awards ($)(2),NaN,...,Total ($),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"David R. Guyer, M.D.(5)",NaN,NaN,2016.0,NaN,NaN,625200,NaN,NaN,—,...,NaN,NaN,325104,NaN,NaN,10405,NaN,NaN,8700989.0,NaN
3,Chief Executive,NaN,NaN,2015.0,NaN,NaN,600000,NaN,NaN,—,...,NaN,NaN,486000,NaN,NaN,18758,NaN,NaN,4866882.0,NaN
4,Officer,NaN,NaN,2014.0,NaN,NaN,545000,NaN,NaN,—,...,NaN,NaN,523200,NaN,NaN,8000,NaN,NaN,5060110.0,NaN
5,"Samir C. Patel, M.D.(6)",NaN,NaN,2016.0,NaN,NaN,591231,NaN,NaN,—,...,NaN,NaN,325104,NaN,NaN,31455,NaN,NaN,8688070.0,NaN
6,Former President,NaN,NaN,2015.0,NaN,NaN,486450,NaN,NaN,—,...,NaN,NaN,361190,NaN,NaN,9038,NaN,NaN,7681275.0,NaN
7,NaN,NaN,NaN,2014.0,NaN,NaN,470000,NaN,NaN,60000,...,NaN,NaN,376000,NaN,NaN,—,NaN,NaN,4093128.0,NaN
8,Glenn P. Sblendorio Sblendorio(7),NaN,NaN,2016.0,NaN,NaN,373154,NaN,NaN,100000,...,NaN,NaN,163350,NaN,NaN,98356,NaN,NaN,8878904.0,NaN
9,President and,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
#Table Formatting tests


SCT = SCTo.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 

for i in range(0,SCT.shape[0]):
    for j in range(0,SCT.shape[1]):
        if type(SCT.iloc[i,j]) == str:
            if SCT.iloc[i,j] == "-" or SCT.iloc[i,j] == "—":
                SCT.iloc[i,j] = 0
            elif '(' in SCT.iloc[i,j]:
                tempstr = SCT.iloc[i,j].split("(")
                tempstr2 = tempstr[0]
                SCT.iloc[i,j] = tempstr2
            elif len(SCT.iloc[i,j]) < 2:
                SCT.iloc[i,j] = np.nan             
                
SCT = SCT.replace('', np.nan)
SCT.iloc[:,0] = SCT.iloc[:,0].fillna(value = '')
SCT = SCT.fillna(value = '')

for i in range(0,SCT.shape[0]):
    if SCT.iloc[i, SCT.shape[1]-1] == '':
        rightslip = ['',SCT.iloc[i]]
        rightslip = rightslip[:-1]
        SCT.iloc[i] = rightslip
    
SCT.replace('',np.nan,inplace = True)
SCT = SCT.dropna()

SCT = SCT.rename(columns=SCT.iloc[0]).drop(index = 1)                

#SCT.apply(filing_table_format)  

#SCT = SCT.stack()
#type(SCT.iloc[3,4])
SCT

,Name and principal position,Year,Salary,Bonus,Stock Awards,Option Awards,Non- Equity Incentive Plan Compensation,All Other Compensation,Total
2,"David R. Guyer, M.D.",2016,625200,0,3291750,4448530,325104,10405,8.70099e+06
3,Chief Executive,2015,600000,0,1048800,2713323,486000,18758,4.86688e+06
4,Officer,2014,545000,0,0,3983910,523200,8000,5.06011e+06
5,"Samir C. Patel, M.D.",2016,591231,0,3291750,4448530,325104,31455,8.68807e+06
6,Former President,2015,486450,0,4511600,2312997,361190,9038,7.68128e+06
8,Glenn P. Sblendorio Sblendorio,2016,373154,100000,3887705,4256339,163350,98356,8.8789e+06
11,Barbara A. Wood,2016,404620,0,632016,899190,129478,17337,2.08264e+06
12,"Senior Vice President,",2015,390940,0,250800,667211,211110,8000,1.52806e+06
13,General Counsel,2014,372311,60000,0,335815,240000,8000,1.01613e+06
14,Henric B. Bjarke,2016,379380,0,695996,529211,121402,97856,1.82384e+06


In [138]:
SCT2 = SCTo.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 
SCT2 = SCTo.apply(SCT_table_format)
SCT2

IndexError: ('tuple index out of range', 'occurred at index 0')